Starting with the file most recent file (2013 - September 10, 2018). Working with Bainbridge departures only.

# Generating 2013

In [1]:
import pandas as pd
import numpy as np

laptop_file_path = 'C:/Users/Keith/Google Drive/H Drive Full Backup/Fall 2018/CS 495 - Senior Seminar/Ferry Data/P000413-090518_2013-2018-09-10.csv'
pc_file_path = "../../Ferry Data/P000413-090518_2013-2018-09-10.csv"
dataset = pd.read_csv(pc_file_path)

bainbridge_departures = dataset.loc[dataset['departure_terminal'] == 'Bainbridge'].reset_index()
bainbridge_departures = bainbridge_departures.drop('index', axis=1)

In [2]:
bainbridge_departures.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
1,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
3,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
4,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


<a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule </a>

In [3]:
# Monday-Friday leave Bainbridge Island Winter 2013
# mfw13 means Monday-Friday Winter 2013
mfw13_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# satw13 means Saturday Winter 2013
satw13_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

# sunw13 means Sunday Winter 2013
sunw13_scheduled_departures_24h = ["05:20:00","07:05:00","08:45:00","9:35:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00","0:00:00","1:25:00"]

#wkw13 means Week Winder 2013
wkw13_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h

In [4]:
def extractMinutesPastHour(time):
    """This takes in a String representing a time in the format 00:mm:ss and returns the int representing mm"""
    m = time.split(':')
    return int(float(m[0]))

def extractMinutes24h(time):
    """This takes in a String representing a time in the format hh:mm:ss and returns the int representing mm"""
    m = time.split(':')
    return int(m[1])

In [5]:
import datetime

def generate_timestamps(dataset, scheduled_departures_24h, starting_point, year, month, day):
    """This method loops through the dataset and matches as many timesamps as it can. Stops and prints 
    an error message if it encounters something odd.

    Keyword arguments:
    dataset -- dataframe with ferry data
    scheduled_departures_24h -- list of all of the departures (represented as strings) for a week
    starting_point -- a pointer to where we should start in the scheduled_departures_24h list
    year -- year of the departure on the top of the dataset
    month -- month of the departure on the top of the dataset
    day -- day of the departure on the top of the dataset
    """

    tdelta = datetime.timedelta(days = 1)

    prevHour = 0

    departure_times = []
    timestamps = []
    weekdays = []

    i = starting_point

    for index, row in dataset.iterrows():
        departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
        hour, minute, seconds = departure24h.split(":")
        scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])
        scheduled_departure_24h = extractMinutes24h(departure24h)

        if(int(prevHour) > int(hour)):
            d = d + tdelta
            year = d.year
            month = d.month
            day = d.day

        if (not scheduled_departure_minutes == scheduled_departure_24h):
            if(d.weekday() == 1):
                # --- this is a quick fix not sure why this works ----
                d = d - tdelta
                year = d.year
                month = d.month
                day = d.day
                # ------------------------------------

                i = i + 1

                departure24h = scheduled_departures_24h[i % len(scheduled_departures_24h)]
                scheduled_hour, scheduled_minutes, scheduled_seconds = departure24h.split(":")
                scheduled_departure_minutes = extractMinutesPastHour(row.iloc[2])

                departure_times.append(departure24h)
                d = datetime.date(year, month, day)
                timestamps.append("%s %02d:%02d:%02d" % (d, int(scheduled_hour), int(scheduled_minutes), int(scheduled_seconds)))
                weekdays.append(d.weekday())

                prevHour = hour  
                i = i + 1
                continue
                if (not scheduled_departure_minutes == scheduled_minutes):
                    print("ERROR")
                    break

            print("""Was able to match departure times up until this row at index %d:\n%s\n
                    Was trying to insert this timestamp: %s""" % 
                                              (index, row,("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))))
            break

        departure_times.append(departure24h)
        d = datetime.date(year, month, day)
        timestamps.append("%s %02d:%02d:%02d" % (d, int(hour), int(minute), int(seconds)))
        weekdays.append(d.weekday())

        prevHour = hour
        i = i + 1
        
    return timestamps

In [6]:
def verify_timestamps(dataset):
    """Loops through the dataset and checks the the minuts past the hour on the timestamp matches 
    minutes past the hour of the scheduled_departure value.

    Keyword arguments:
    dataset - DataFrame with ferry data indexed with timestamps
    """
    for index, row in dataset.iterrows():
        scheduled_minute = index.minute
        dataset_minute = extractMinutesPastHour(row.iloc[2])
        
        if(not scheduled_minute == dataset_minute):    
            print("""Was able to match departure times up until this row at index %s:\n%s""" % 
                                              (index, row))
            return
            
    print("Success!")
    
def verify_arrival_terminals(dataset):
    """The only passenger sailing departing Bainbridge goes to Seattle. I was noticing that there were
    some other odd arrival terminals (seems like maintence stuff). This finds those terminals and creates
    a with the row entries that have odd arrival terminals. This is so the user can visually see the odd terminals.

    Keyword arguments:
    dataset - DataFrame with ferry data indexed with timestamps
    """
    odd_terminals = []
    
    for index, row in dataset.iterrows():
        arrival_terminal = str(row.iloc[1])
        
        if(not (arrival_terminal == "Colman")):
            if(not arrival_terminal == "nan"):
                odd_terminals.append(row)
    
    return pd.DataFrame(odd_terminals)

In [7]:
def remove_odd_terminals(dataset, odd_terminals):
    """Removes the odd_terminals (arrival terminals) from the main dataset.

    Keyword arguments:
    dataset - DataFrame with ferry data indexed by timestamps
    odd_terminals - DataFrame with all the entires that have odd arrival terminals.
    """
    return dataset[~dataset.index.isin(odd_terminals.index)]

In [8]:
timestamps = generate_timestamps(bainbridge_departures, wkw13_scheduled_departures_24h, 23, 2013, 1, 1)

Was able to match departure times up until this row at index 3297:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:30.0
Actual_Arrival            36:59.0
TransitTime                    31
Name: 3297, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 05:20:00


In [9]:
len(timestamps)

3297

In [10]:
timestamps[3290:]

['2013-05-26 20:10:00',
 '2013-05-26 20:55:00',
 '2013-05-26 21:45:00',
 '2013-05-26 22:30:00',
 '2013-05-27 00:00:00',
 '2013-05-27 01:25:00',
 '2013-05-27 04:45:00']

Want to git rid of those last three dates.

In [11]:
timestamps = timestamps[:3296]

In [12]:
len(timestamps)

3296

May 27, 2013 Starts at index 3296 in `bainbridge_departures`

In [13]:
bainbridge_departures.iloc[3296:3300]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3296,Bainbridge,Colman,20:00.0,20:11.0,51:37.0,31.0
3297,Bainbridge,Colman,05:00.0,05:30.0,36:59.0,31.0
3298,Bainbridge,Colman,55:00.0,56:22.0,26:55.0,30.0
3299,Bainbridge,Colman,45:00.0,45:00.0,16:17.0,31.0


The only difference between the <a href = "https://web.archive.org/web/20130123233819/http:/www.wsdot.wa.gov/ferries/pdf/2013winter.pdf"> Winter 2013 Schedule</a> and the <a href = "https://web.archive.org/web/20130624071202/http:/www.wsdot.wa.gov/ferries/pdf/2013Spring.pdf"> Spring 2013 Schedule</a> happend on May 27, 2013

In [14]:
# May 27, 2013
may2713 = ["05:20:00","07:05:00","07:55:00","08:45:00","9:35:00", "10:25:00","11:30:00","12:20:00","13:10:00",
           "14:05:00","14:55:00","15:50:00","16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
           "21:45:00","22:30:00","0:00:00","1:25:00"]

In [15]:
may2713timestamps = generate_timestamps(bainbridge_departures.iloc[3296:], may2713, 0, 2013, 5, 27)

Was able to match departure times up until this row at index 3319:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:00.0
Actual_Arrival            51:12.0
TransitTime                    31
Name: 3319, dtype: object

                    Was trying to insert this timestamp: 2013-05-27 07:55:00


In [16]:
may2713timestamps = may2713timestamps[:22]

In [17]:
timestamps = timestamps + may2713timestamps

This below is the beginning of Tuesday, May 28, 2013

In [18]:
bainbridge_departures.iloc[3318:3322]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
3318,Bainbridge,Colman,45:00.0,45:56.0,16:44.0,31.0
3319,Bainbridge,Colman,20:00.0,20:00.0,51:12.0,31.0
3320,Bainbridge,Colman,20:00.0,23:59.0,51:35.0,28.0
3321,Bainbridge,Colman,05:00.0,06:51.0,40:22.0,34.0


In [19]:
len(mfw13_scheduled_departures_24h)

23

In [20]:
timestamps_2 = generate_timestamps(bainbridge_departures.iloc[3318:], wkw13_scheduled_departures_24h, 24, 2013, 5, 28)

Was able to match departure times up until this row at index 4154:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          19:33.0
Actual_Arrival            49:12.0
TransitTime                    30
Name: 4154, dtype: object

                    Was trying to insert this timestamp: 2013-07-04 04:45:00


In [21]:
timestamps = timestamps + timestamps_2

In [22]:
len(timestamps)

4154

In [23]:
timestamps[4150:]

['2013-07-03 20:55:00',
 '2013-07-03 21:45:00',
 '2013-07-03 22:35:00',
 '2013-07-04 00:55:00']

The fourth of July schedule is the same as the May 27, 2013 schedule.

Thursday, July 4, 2013

In [24]:
july0413timestamps = generate_timestamps(bainbridge_departures[4154:], may2713, 0, 2013, 7, 4)

Was able to match departure times up until this row at index 4176:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          44:36.0
Actual_Arrival            13:56.0
TransitTime                    29
Name: 4176, dtype: object

                    Was trying to insert this timestamp: 2013-07-05 05:20:00


In [25]:
timestamps = timestamps + july0413timestamps

In [26]:
len(timestamps)

4176

Need to start again this time from Thursday

In [27]:
timestamps_2 = generate_timestamps(bainbridge_departures[4176:], wkw13_scheduled_departures_24h, (23*4) + 1, 2013, 7, 5)

Was able to match departure times up until this row at index 5507:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       05:00.0
actual_departure          05:06.0
Actual_Arrival            32:58.0
TransitTime                    27
Name: 5507, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 05:20:00


In [28]:
timestamps = timestamps + timestamps_2

Monday, September 2, 2013 was a holiday.

In [29]:
timestamps = timestamps[:5506]

In [30]:
len(timestamps)

5506

In [31]:
timestamps[5500:]

['2013-09-01 20:10:00',
 '2013-09-01 20:55:00',
 '2013-09-01 21:45:00',
 '2013-09-01 22:30:00',
 '2013-09-02 00:00:00',
 '2013-09-02 01:25:00']

In [32]:
bainbridge_departures.iloc[5506:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5506,Bainbridge,Colman,20:00.0,20:16.0,49:50.0,29.0
5507,Bainbridge,Colman,05:00.0,05:06.0,32:58.0,27.0
5508,Bainbridge,Colman,55:00.0,55:00.0,26:35.0,31.0
5509,Bainbridge,Colman,45:00.0,45:06.0,14:50.0,29.0
5510,Bainbridge,Colman,35:00.0,35:36.0,06:01.0,31.0


In [33]:
sept0213timestamps = generate_timestamps(bainbridge_departures[5506:], may2713, 0, 2013, 9, 2)

Was able to match departure times up until this row at index 5529:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:05.0
Actual_Arrival            53:03.0
TransitTime                    33
Name: 5529, dtype: object

                    Was trying to insert this timestamp: 2013-09-02 07:55:00


In [34]:
sept0213timestamps = sept0213timestamps[:22]

In [35]:
timestamps = timestamps + sept0213timestamps

In [36]:
len(timestamps)

5528

In [37]:
timestamps[5520:]

['2013-09-02 18:30:00',
 '2013-09-02 19:10:00',
 '2013-09-02 20:10:00',
 '2013-09-02 20:55:00',
 '2013-09-02 21:45:00',
 '2013-09-02 22:30:00',
 '2013-09-03 00:00:00',
 '2013-09-03 01:25:00']

In [38]:
bainbridge_departures.iloc[5528:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
5528,Bainbridge,Colman,45:00.0,45:07.0,12:15.0,27.0
5529,Bainbridge,Colman,20:00.0,20:05.0,53:03.0,33.0
5530,Bainbridge,Colman,20:00.0,20:00.0,49:57.0,29.0
5531,Bainbridge,Colman,05:00.0,06:03.0,36:39.0,30.0
5532,Bainbridge,Colman,55:00.0,56:43.0,26:53.0,30.0


Tuesday, September 3, 2013 (is the start of whats above)

In [39]:
bainbridge_departures = bainbridge_departures.drop(7185)

In [40]:
timestamps_2 = generate_timestamps(bainbridge_departures[5528:], wkw13_scheduled_departures_24h, (23*1) + 1, 2013, 9, 3)

Was able to match departure times up until this row at index 7471:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          21:09.0
Actual_Arrival            52:53.0
TransitTime                    31
Name: 7471, dtype: object

                    Was trying to insert this timestamp: 2013-11-28 04:45:00


In [41]:
len(timestamps_2)

1942

In [42]:
timestamps = timestamps + timestamps_2

In [43]:
timestamps_2[1935:]

['2013-11-27 18:30:00',
 '2013-11-27 19:10:00',
 '2013-11-27 20:10:00',
 '2013-11-27 20:55:00',
 '2013-11-27 21:45:00',
 '2013-11-27 22:35:00',
 '2013-11-28 00:55:00']

In [44]:
bainbridge_departures[7470:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
7471,Bainbridge,Colman,20:00.0,21:09.0,52:53.0,31.0
7472,Bainbridge,Colman,05:00.0,04:41.0,41:21.0,37.0
7473,Bainbridge,Colman,55:00.0,54:33.0,25:12.0,31.0
7474,Bainbridge,Colman,45:00.0,44:43.0,20:07.0,36.0
7475,Bainbridge,Colman,35:00.0,35:00.0,04:01.0,29.0


In [45]:
nov2813timestamps = generate_timestamps(bainbridge_departures[7470:], may2713, 0, 2013, 11, 28)

Was able to match departure times up until this row at index 7493:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          45:58.0
Actual_Arrival            20:01.0
TransitTime                    35
Name: 7493, dtype: object

                    Was trying to insert this timestamp: 2013-11-29 05:20:00


In [46]:
timestamps = timestamps + nov2813timestamps

In [47]:
bainbridge_departures[7492:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
7493,Bainbridge,Colman,45:00.0,45:58.0,20:01.0,35.0
7494,Bainbridge,Colman,20:00.0,20:03.0,51:11.0,31.0
7495,Bainbridge,Colman,20:00.0,22:03.0,52:19.0,30.0
7496,Bainbridge,Colman,05:00.0,05:01.0,35:29.0,30.0
7497,Bainbridge,Colman,55:00.0,55:26.0,26:27.0,31.0


The top of the above is Friday, November 29, 2013

In [48]:
timestamps_2 = generate_timestamps(bainbridge_departures[7492:], wkw13_scheduled_departures_24h, (23*4) + 1, 2013, 11, 29)

Was able to match departure times up until this row at index 8079:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:15.0
Actual_Arrival            51:40.0
TransitTime                    31
Name: 8079, dtype: object

                    Was trying to insert this timestamp: 2013-12-25 04:45:00


In [49]:
timestamps = timestamps + timestamps_2

In [50]:
bainbridge_departures[8078:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8079,Bainbridge,Colman,20:00.0,20:15.0,51:40.0,31.0
8080,Bainbridge,Colman,05:00.0,06:36.0,38:42.0,32.0
8081,Bainbridge,Colman,55:00.0,55:25.0,24:45.0,29.0
8082,Bainbridge,Colman,45:00.0,45:00.0,15:54.0,30.0
8083,Bainbridge,Colman,35:00.0,37:32.0,11:13.0,34.0


In [51]:
dec2513timestamps = generate_timestamps(bainbridge_departures[8078:], may2713, 0, 2013, 12, 25)

Was able to match departure times up until this row at index 8101:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          44:36.0
Actual_Arrival            14:25.0
TransitTime                    30
Name: 8101, dtype: object

                    Was trying to insert this timestamp: 2013-12-26 05:20:00


In [52]:
timestamps = timestamps + dec2513timestamps

In [53]:
bainbridge_departures[8100:].head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8101,Bainbridge,Colman,45:00.0,44:36.0,14:25.0,30.0
8102,Bainbridge,Colman,20:00.0,20:53.0,50:24.0,30.0
8103,Bainbridge,Colman,20:00.0,19:40.0,50:15.0,31.0
8104,Bainbridge,Colman,05:00.0,06:06.0,35:48.0,29.0
8105,Bainbridge,Colman,55:00.0,55:29.0,26:33.0,31.0


The top of above is Thursday December 26, 2013

In [54]:
timestamps_2 = generate_timestamps(bainbridge_departures[8100:], wkw13_scheduled_departures_24h, (23*3) + 1, 2013, 12, 26)

Was able to match departure times up until this row at index 8236:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       20:00.0
actual_departure          20:00.0
Actual_Arrival            55:34.0
TransitTime                    35
Name: 8236, dtype: object

                    Was trying to insert this timestamp: 2014-01-01 04:45:00


In [55]:
timestamps = timestamps + timestamps_2

In [56]:
len(timestamps)

8235

In [57]:
bainbridge_departures_2013_with_index = bainbridge_departures[0:8235]
bainbridge_departures_2013_with_index.index = pd.to_datetime(timestamps)

In [58]:
bainbridge_departures_2013_with_index = bainbridge_departures_2013_with_index["2013"]

In [59]:
bainbridge_departures_2013_with_index.index.name = "timestamps"

In [60]:
verify_timestamps(bainbridge_departures_2013_with_index)

Success!


In [61]:
odd_terminals = verify_arrival_terminals(bainbridge_departures_2013_with_index)
odd_terminals

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2013-06-03 17:30:00,Bainbridge,Pier 54 - Standby,30:00.0,NaN,03:02.0,NaN
2013-06-21 14:05:00,Bainbridge,Orcas,05:00.0,NaN,17:22.0,NaN
2013-09-18 09:40:00,Bainbridge,Pier 54 - Standby,40:00.0,NaN,12:12.0,NaN


In [62]:
bainbridge_departures_2013_with_index = remove_odd_terminals(bainbridge_departures_2013_with_index, odd_terminals)

In [63]:
bainbridge_departures_2013_with_index.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
timestamps,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


In [64]:
bainbridge_departures_2013_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
timestamps,,,,,,
2013-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:03.0,44:26.0,34.0
2013-12-31 20:10:00,Bainbridge,Colman,10:00.0,10:53.0,41:57.0,31.0
2013-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:36.0,31.0
2013-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:17.0,17:09.0,32.0
2013-12-31 22:35:00,Bainbridge,Colman,35:00.0,38:45.0,13:36.0,35.0


In [65]:
bainbridge_departures_2013_with_index.shape

(8231, 6)

In [66]:
timestamps_2013 = bainbridge_departures_2013_with_index.index

# Generating 2014

<a href = "https://stackoverflow.com/questions/48796729/how-to-add-a-year-to-a-column-of-dates-in-pandas">How to add a year to a column of dates in pandas </a>

In [67]:
timestamps_2014 = timestamps_2013 + pd.offsets.DateOffset(years=1)

The Holdiays in 2014:
- Wednesday, January 1, 2014
- Monday, May 26, 2014
- Friday, July 4, 2014
- Monday, September 1, 2014
- Thursday, November 27, 2014
- Thursday, December 25, 2014

`d.weekday()`: <a href = "https://docs.python.org/2/library/datetime.html">Monday is 0 and Sunday is 6</a>

<a href = "https://stackoverflow.com/questions/17152719/change-date-of-a-datetimeindex">Change date of a DateTimeIndex</a>

<a href = "https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists">How to make a flat list out of list of lists?</a>

In [68]:
from collections import OrderedDict
def generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday, firstEntries = False):
    """This method is able to generate more timestamps given the past years timestamps. Generally, the schedule
    does not change much so we can take a previous years timestamps and just move around the holidays and add one year
    and that should be the next years timestamps.

    Keyword arguments:
    startingDate -- datetime object representing the starting date of our new timestamps to be generated
    holidays -- a list with the holiday departure schedule
    weekday -- a list with the weekday departure schedule
    saturday -- a list with the saturday departure schedule
    sunday -- a list with the sunday departure schedule
    firstEntries -- a list with the entries to start out list out with. This is because the schedule includes
    next day departures on a previous days schedules. (ex on the 'Sunday' schedule there might be a 12:05 departure
    since 12:05 is past midnight this actually departs on Monday but it only happens right after Sunday.)
    """
    if(not firstEntries):
        timestamps = [[pd.Timestamp('%s-01-01 00:55:00' % (startingDate.year))]]
    else:
        timestamps = [firstEntries]
        
    startingYear = startingDate.year
    d = startingDate
    
    
    while(not (d.year == (startingYear + 1))):
        #pdb.set_trace()
        if(d in holidays):
            timestamps.append(saturday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
            temp = d + datetime.timedelta(days = 1)
            timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
        else:
            if(d.weekday() == 5):
                timestamps.append(saturday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
            elif(d.weekday() == 6):
                timestamps.append(sunday[2:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(sunday[0:2].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
            else:
                timestamps.append(weekday[1:].map(lambda t: t.replace(year=d.year, month=d.month, day=d.day)).tolist())
                temp = d + datetime.timedelta(days = 1)
                timestamps.append(weekday[0:1].map(lambda t: t.replace(year=temp.year, month=temp.month, day=temp.day)).tolist())
        
        d = d + datetime.timedelta(days = 1)
        
    return list(OrderedDict.fromkeys([item for sublist in timestamps for item in sublist]))

In [69]:
holidays = [datetime.date(2014, 1, 1), datetime.date(2014, 5, 26), datetime.date(2014, 7, 4), datetime.date(2014, 9, 1),
           datetime.date(2014, 11, 27), datetime.date(2014, 12, 25)]
weekday = bainbridge_departures_2013_with_index["2013-01-02"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2013_with_index["2013-01-05"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2013_with_index["2013-01-06"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2014, 1, 1)

In [70]:
timestamps_2014 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday)

In [71]:
len(timestamps_2014)

8234

In [72]:
# Dropping 2013
bainbridge_departures_n13 = bainbridge_departures[8234:]

In [73]:
bainbridge_departures_n13 = bainbridge_departures_n13.reset_index(drop=True)

In [74]:
bainbridge_departures_n13.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,13:39.0,46:53.0,33.0
1,Bainbridge,Colman,20:00.0,20:00.0,55:34.0,35.0
2,Bainbridge,Colman,05:00.0,04:34.0,35:51.0,31.0
3,Bainbridge,Colman,55:00.0,55:00.0,26:14.0,31.0
4,Bainbridge,Colman,45:00.0,45:00.0,14:54.0,29.0


In [75]:
bainbridge_departures_2014_with_index = bainbridge_departures_n13[:8235]
bainbridge_departures_2014_with_index.tail()
#bainbridge_departures_2013_with_index.index = pd.to_datetime(timestamps_2014)

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
8230,Bainbridge,Colman,10:00.0,11:05.0,41:10.0,30.0
8231,Bainbridge,Colman,55:00.0,55:10.0,26:03.0,31.0
8232,Bainbridge,Colman,45:00.0,47:13.0,19:07.0,32.0
8233,Bainbridge,Colman,35:00.0,39:34.0,11:57.0,32.0
8234,Bainbridge,Colman,55:00.0,05:15.0,36:38.0,31.0


In [76]:
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index.drop(2763)
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index.reset_index(drop=True)

In [77]:
len(timestamps_2014)

8234

In [78]:
bainbridge_departures_2014_with_index.shape

(8234, 6)

In [79]:
bainbridge_departures_2014_with_index.index = pd.to_datetime(timestamps_2014)
bainbridge_departures_2014_with_index = bainbridge_departures_2014_with_index["2014"]

In [80]:
bainbridge_departures_2014_with_index.index.name = "timestamps"

In [81]:
verify_timestamps(bainbridge_departures_2014_with_index)

Success!


In [82]:
odd_terminals = verify_arrival_terminals(bainbridge_departures_2014_with_index)
odd_terminals

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2014-05-31 10:25:00,Bainbridge,Todd Shipyard (2),25:00.0,NaN,35:03.0,NaN
2014-09-03 21:45:00,Bainbridge,Pier 54 - Standby,45:00.0,NaN,21:54.0,NaN


In [83]:
bainbridge_departures_2014_with_index = remove_odd_terminals(bainbridge_departures_2014_with_index, odd_terminals)

In [84]:
bainbridge_departures_2014_with_index.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
timestamps,,,,,,
2014-01-01 00:55:00,Bainbridge,Colman,55:00.0,13:39.0,46:53.0,33.0
2014-01-01 05:20:00,Bainbridge,Colman,20:00.0,20:00.0,55:34.0,35.0
2014-01-01 07:05:00,Bainbridge,Colman,05:00.0,04:34.0,35:51.0,31.0
2014-01-01 07:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:14.0,31.0
2014-01-01 08:45:00,Bainbridge,Colman,45:00.0,45:00.0,14:54.0,29.0


In [85]:
bainbridge_departures_2014_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
timestamps,,,,,,
2014-12-31 19:10:00,Bainbridge,Colman,10:00.0,16:19.0,48:52.0,32.0
2014-12-31 20:10:00,Bainbridge,Colman,10:00.0,11:05.0,41:10.0,30.0
2014-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:10.0,26:03.0,31.0
2014-12-31 21:45:00,Bainbridge,Colman,45:00.0,47:13.0,19:07.0,32.0
2014-12-31 22:35:00,Bainbridge,Colman,35:00.0,39:34.0,11:57.0,32.0


In [86]:
bainbridge_departures_2014_with_index.index.name = "timestamps"

In [87]:
bainbridge_departures_n13.iloc[8234]

departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       55:00.0
actual_departure          05:15.0
Actual_Arrival            36:38.0
TransitTime                    31
Name: 8234, dtype: object

In [88]:
bainbridge_departures_n13_n14 = bainbridge_departures_n13.iloc[8234:].reset_index(drop=True)

# Generating 2015

Here is the start of 2015!

In [89]:
bainbridge_departures_n13_n14.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,05:15.0,36:38.0,31.0
1,Bainbridge,Colman,20:00.0,21:48.0,51:23.0,30.0
2,Bainbridge,Colman,05:00.0,06:20.0,36:29.0,30.0
3,Bainbridge,Colman,55:00.0,55:00.0,27:16.0,32.0
4,Bainbridge,Colman,45:00.0,45:55.0,15:02.0,30.0


The Holdiays in 2015:
- Thursday, January 1, 2015
- Monday, May 25, 2015
- Saturday, July 4, 2015
- Monday, September 7, 2015
- Thursday, November 26, 2015
- Friday, December 25, 2015

In [90]:
holidays = [datetime.date(2015, 1, 1), datetime.date(2015, 5, 25), datetime.date(2015, 7, 4), datetime.date(2015, 9, 7),
           datetime.date(2015, 11, 26), datetime.date(2015, 12, 25)]
weekday = bainbridge_departures_2013_with_index["2013-01-02"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2013_with_index["2013-01-05"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2013_with_index["2013-01-06"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2015, 1, 1)

In [91]:
timestamps_2015 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday)

In [92]:
len(timestamps_2015)

8235

In [93]:
timestamps_2015[8234]

Timestamp('2016-01-01 00:55:00')

In [94]:
# Only want 2015 not the first entry of 2016
timestamps_2015 = timestamps_2015[:8234]

In [95]:
bainbridge_departures_2015_with_index = bainbridge_departures_n13_n14[:8234].reset_index(drop=True)
bainbridge_departures_2015_with_index.index = pd.to_datetime(timestamps_2015)
bainbridge_departures_2015_with_index = bainbridge_departures_2015_with_index["2015"]

In [96]:
verify_timestamps(bainbridge_departures_2015_with_index)

Success!


In [97]:
odd_terminals = verify_arrival_terminals(bainbridge_departures_2015_with_index)
odd_terminals

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2015-01-01 11:30:00,Bainbridge,Lake Union Shipyard,30:00.0,NaN,45:06.0,NaN
2015-01-29 14:55:00,Bainbridge,Fuel Dock - Pier 15,55:00.0,NaN,14:27.0,NaN
2015-03-20 16:35:00,Bainbridge,Pier 54 - Standby,35:00.0,NaN,20:38.0,NaN


In [98]:
bainbridge_departures_2015_with_index = remove_odd_terminals(bainbridge_departures_2015_with_index, odd_terminals)

In [99]:
bainbridge_departures_2015_with_index.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2015-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:15.0,36:38.0,31.0
2015-01-01 05:20:00,Bainbridge,Colman,20:00.0,21:48.0,51:23.0,30.0
2015-01-01 07:05:00,Bainbridge,Colman,05:00.0,06:20.0,36:29.0,30.0
2015-01-01 07:55:00,Bainbridge,Colman,55:00.0,55:00.0,27:16.0,32.0
2015-01-01 08:45:00,Bainbridge,Colman,45:00.0,45:55.0,15:02.0,30.0


In [100]:
len(bainbridge_departures_2015_with_index)

8231

In [101]:
bainbridge_departures_n13_n14_n15 = bainbridge_departures_n13_n14[8234:].reset_index(drop=True)

# Generating 2016

Here is the start of 2016!

In [102]:
bainbridge_departures_n13_n14_n15.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,55:00.0,15:15.0,45:51.0,30.0
1,Bainbridge,Colman,20:00.0,21:45.0,54:24.0,33.0
2,Bainbridge,Colman,05:00.0,05:49.0,39:03.0,34.0
3,Bainbridge,Colman,55:00.0,56:22.0,27:20.0,31.0
4,Bainbridge,Colman,45:00.0,45:42.0,15:53.0,30.0


The Holdiays in 2016:
- Friday, January 1, 2016
- Monday, May 30, 2016
- Monday, July 4, 2016
- Monday, September 5, 2016
- Thursday, November 24, 2016
- Sunday, December 25, 2016

<i>Note that since Christmas is on a Sunday December 26 is considered a holiday.</i>

In [103]:
holidays = [datetime.date(2016, 1, 1), datetime.date(2016, 5, 30), datetime.date(2016, 7, 4), datetime.date(2016, 9, 5),
           datetime.date(2016, 11, 24), datetime.date(2016, 12, 26)]
weekday = bainbridge_departures_2013_with_index["2013-01-02"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2013_with_index["2013-01-05"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2013_with_index["2013-01-06"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2016, 1, 1)

In [104]:
timestamps_2016 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday)

In [105]:
len(timestamps_2016)

8256

In [106]:
bainbridge_departures_2016_with_index = bainbridge_departures_n13_n14_n15[:8256].reset_index(drop=True)
bainbridge_departures_2016_with_index.index = pd.to_datetime(timestamps_2016)
bainbridge_departures_2016_with_index = bainbridge_departures_2016_with_index["2016"]

In [107]:
verify_timestamps(bainbridge_departures_2016_with_index)

Was able to match departure times up until this row at index 2016-06-12 09:35:00:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          46:38.0
Actual_Arrival            18:28.0
TransitTime                    32
Name: 2016-06-12 09:35:00, dtype: object


The <a href = "https://web.archive.org/web/20160617152902/http:/www.wsdot.wa.gov/ferries/pdf/2016Summer.pdf">Summer of 2016</a> had a schedule change in it. This starts in June 12, 2016

In [108]:
timestamps_2016 = bainbridge_departures_2016_with_index[:"2016-06-11"].index.tolist()

In [109]:
# Monday-Friday leave Bainbridge Island Summer 2016
# mfw16 means Monday-Friday Summer 2016
mfs16_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:10:00", "14:05:00", "14:55:00", "15:50:00", "16:35:00", 
                            "17:30:00", "18:30:00", "19:10:00", "20:10:00", "20:55:00", "21:45:00", "22:35:00"]

# satw16 means Saturday Summer 2016
sats16_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:45:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00"]

# sunw16 means Sunday Summer 2016
suns16_scheduled_departures_24h = ["0:00:00", "1:25:00", "05:20:00","07:05:00","08:45:00","9:45:00", "10:25:00",
                                  "11:30:00","12:20:00","13:10:00","14:05:00","14:55:00","15:50:00",
                                  "16:35:00","17:30:00","18:30:00","19:10:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00"]

#wks16 means Week Summer 2016
wks16_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h

`stringsToDatetime` takes in something like 

`["05:20:00","07:05:00","08:45:00"]` 

and returns 

`[datetime.datetime(1900, 1, 1, 5, 20), datetime.datetime(1900, 1, 1, 7, 5), datetime.datetime(1900, 1, 1, 8, 45)]`

In [110]:
def stringsToDatetime(strings):
    """Turns a list of strings that represent 24h time and turns them into datetime object.

    Keyword arguments:
    strings -- list of strings representing a 24h time.
    """
    return pd.to_datetime([datetime.datetime.strptime(x, '%H:%M:%S') for x in strings])

In [111]:
holidays = [datetime.date(2016, 1, 1), datetime.date(2016, 5, 30), datetime.date(2016, 7, 4), datetime.date(2016, 9, 5),
           datetime.date(2016, 11, 24), datetime.date(2016, 12, 26)]
weekday = stringsToDatetime(mfs16_scheduled_departures_24h)
saturday = stringsToDatetime(sats16_scheduled_departures_24h)
sunday = stringsToDatetime(suns16_scheduled_departures_24h)

startingDate = datetime.date(2016, 6, 12)

In [112]:
timestamps_2016_2 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday,
                                             [pd.Timestamp(2016, 6, 12, 0, 0), pd.Timestamp(2016, 6, 12, 1, 25)])

In [113]:
timestamps_2016 = timestamps_2016 + timestamps_2016_2

In [114]:
len(timestamps_2016)

8256

In [115]:
bainbridge_departures_2016_with_index = bainbridge_departures_n13_n14_n15[:8256].reset_index(drop=True)
bainbridge_departures_2016_with_index.index = pd.to_datetime(timestamps_2016)
bainbridge_departures_2016_with_index = bainbridge_departures_2016_with_index["2016"]

In [116]:
verify_timestamps(bainbridge_departures_2016_with_index)

Success!


In [117]:
odd_terminals = verify_arrival_terminals(bainbridge_departures_2016_with_index)
odd_terminals

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2016-10-02 18:30:00,Bainbridge,Eagle Harbor,30:00.0,NaN,43:26.0,NaN


In [118]:
bainbridge_departures_2016_with_index = remove_odd_terminals(bainbridge_departures_2016_with_index, odd_terminals)

In [119]:
bainbridge_departures_2016_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2016-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:00.0,41:26.0,31.0
2016-12-31 20:10:00,Bainbridge,Colman,10:00.0,09:40.0,41:45.0,32.0
2016-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,25:50.0,30.0
2016-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:19.0,19:24.0,34.0
2016-12-31 22:30:00,Bainbridge,Colman,30:00.0,31:11.0,03:02.0,32.0


In [120]:
len(bainbridge_departures_2016_with_index)

8253

In [121]:
bainbridge_departures_n13_n14_n15_n16 = bainbridge_departures_n13_n14_n15[8254:].reset_index(drop=True)

# Generating 2017

Here is the start of 2017!

In [122]:
bainbridge_departures_n13_n14_n15_n16.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,00:00.0,59:34.0,30:25.0,31.0
1,Bainbridge,Colman,25:00.0,33:50.0,06:40.0,33.0
2,Bainbridge,Colman,20:00.0,20:52.0,52:31.0,32.0
3,Bainbridge,Colman,05:00.0,06:03.0,35:15.0,29.0
4,Bainbridge,Colman,45:00.0,44:31.0,15:34.0,31.0


The Holdiays in 2017:
- Monday, January 2, 2017 (Since Jan 1 is a Sunday)
- Monday, May 29, 2017
- Tuesday, July 4, 2017
- Monday, September 4, 2017
- Thursday, November 23, 2017
- Monday, December 25, 2016

Note: was originally have issues with:

`weekday = bainbridge_departures_2016_with_index["2016-01-04"].index + pd.offsets.DateOffset(years=1)`

This is because January 4, 2016 is right after a weekend so the past midnight sailings from the Sunday schedule carry over and we do not want that for a general weekday. In the future always remeber to choose a weekday the proceeds a regular weekday and not a holiday or weekend.

In [123]:
holidays = [datetime.date(2017, 1, 2), datetime.date(2017, 5, 29), datetime.date(2017, 7, 4), datetime.date(2017, 9, 4),
           datetime.date(2017, 11, 23), datetime.date(2017, 12, 25)]
weekday = bainbridge_departures_2016_with_index["2016-01-05"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2016_with_index["2016-07-9"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2016_with_index["2016-07-10"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2017, 1, 1)

In [124]:
timestamps_2017 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday,
                                          [pd.Timestamp(2017, 1, 1, 0, 0), pd.Timestamp(2017, 1, 1, 1, 25)])

In [125]:
len(timestamps_2017)

8233

In [126]:
bainbridge_departures_2017_with_index = bainbridge_departures_n13_n14_n15_n16[:8233].reset_index(drop=True)
bainbridge_departures_2017_with_index.index = pd.to_datetime(timestamps_2017)
bainbridge_departures_2017_with_index = bainbridge_departures_2017_with_index["2017"]

In [127]:
verify_timestamps(bainbridge_departures_2017_with_index)

Success!


In [128]:
odd_terminals = verify_arrival_terminals(bainbridge_departures_2017_with_index)
odd_terminals

""


In [129]:
bainbridge_departures_2017_with_index.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2017-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:00.0,44:24.0,34.0
2017-12-31 20:10:00,Bainbridge,Colman,10:00.0,10:00.0,42:46.0,32.0
2017-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,27:58.0,32.0
2017-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:00.0,16:20.0,31.0
2017-12-31 22:30:00,Bainbridge,Colman,30:00.0,30:00.0,03:56.0,33.0


In [130]:
len(bainbridge_departures_2017_with_index)

8231

In [131]:
bainbridge_departures_n13_n14_n15_n16_n17 = bainbridge_departures_n13_n14_n15_n16[8231:].reset_index(drop=True)

# Generating 2018

Here is the start of 2018!

In [132]:
bainbridge_departures_n13_n14_n15_n16_n17.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
0,Bainbridge,Colman,00:00.0,00:00.0,31:23.0,31.0
1,Bainbridge,Colman,25:00.0,40:26.0,13:57.0,33.0
2,Bainbridge,Colman,20:00.0,20:30.0,54:45.0,34.0
3,Bainbridge,Colman,05:00.0,05:12.0,39:06.0,34.0
4,Bainbridge,Colman,55:00.0,55:00.0,24:34.0,29.0


The Holdiays in 2018:
- Monday, January 1, 2018
- Monday, May 28, 2018
- Wednesday, July 4, 2018
- Monday, September 3, 2018
- Thursday, November 22, 2018
- Tuesday, December 25, 2018

In [133]:
holidays = [datetime.date(2018, 1, 1), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4), datetime.date(2018, 9, 3),
           datetime.date(2018, 11, 22), datetime.date(2018, 12, 25)]
weekday = bainbridge_departures_2016_with_index["2016-01-5"].index + pd.offsets.DateOffset(years=1)
saturday = bainbridge_departures_2016_with_index["2016-07-9"].index + pd.offsets.DateOffset(years=1)
sunday = bainbridge_departures_2016_with_index["2016-07-10"].index + pd.offsets.DateOffset(years=1)

startingDate = datetime.date(2018, 1, 1)

In [134]:
timestamps_2018 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday,
                                          [pd.Timestamp(2018, 1, 1, 0, 0), pd.Timestamp(2018, 1, 1, 1, 25)])

In [135]:
len(bainbridge_departures_n13_n14_n15_n16_n17)

5726

In [136]:
timestamps_2018 = timestamps_2018[:5726]

In [137]:
bainbridge_departures_2018_with_index = bainbridge_departures_n13_n14_n15_n16_n17.reset_index(drop=True)
bainbridge_departures_2018_with_index.index = pd.to_datetime(timestamps_2018)
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index["2018"]

In [138]:
verify_timestamps(bainbridge_departures_2018_with_index)

Was able to match departure times up until this row at index 2018-01-07 11:30:00:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       35:00.0
actual_departure          38:42.0
Actual_Arrival            10:41.0
TransitTime                    32
Name: 2018-01-07 11:30:00, dtype: object


In [139]:
timestamps_2018 = bainbridge_departures_2018_with_index[:"2018-01-6"].index.tolist()

The <a href = "https://web.archive.org/web/20180127104841/http://www.wsdot.wa.gov/ferries/pdf/2018Winter.pdf">Winter of 2018</a> had a schedule change in it. This starts in January 7, 2018

In [140]:
# Monday-Friday leave Bainbridge Island Winter 2018
# mfw18 means Monday-Friday Winter 2018
mfw18_scheduled_departures_24h = ["0:55:00", "4:45:00", "5:20:00", "6:20:00", "7:05:00", "7:55:00","8:45:00", "9:40:00", 
                            "10:25:00", "11:30:00", "12:20:00", "13:15:00", "14:05:00", "14:55:00", "15:50:00", "16:45:00", "17:35:00", 
                            "18:40:00", "19:20:00", "20:20:00", "21:00:00", "21:55:00", "23:35:00"]

# satw18 means Saturday Winter 2018
satw18_scheduled_departures_24h = ["0:55:00","05:20:00","07:05:00","07:55:00","08:45:00","9:45:00", "10:25:00",
                                  "11:35:00","12:20:00","13:20:00","14:10:00","15:00:00","15:55:00", "16:40:00",
                                  "17:35:00","18:30:00","19:15:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00"]

# sunw18 means Sunday Winter 2018
sunw18_scheduled_departures_24h = ["0:00:00", "01:25:00", "05:20:00","07:05:00","08:45:00","9:45:00", "10:25:00",
                                  "11:35:00","12:20:00","13:20:00","14:10:00","15:00:00","15:55:00", "16:40:00",
                                  "17:35:00","18:30:00","19:15:00","20:10:00","20:55:00",
                                  "21:45:00","22:30:00"]

#wkw18 means Week Winter 2018
wkw18_scheduled_departures_24h = (5*mfw13_scheduled_departures_24h) + satw13_scheduled_departures_24h + sunw13_scheduled_departures_24h



In [141]:
#holidays = [datetime.date(2018, 1, 1), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4), datetime.date(2018, 9, 3),
#           datetime.date(2018, 11, 22), datetime.date(2018, 12, 25)]
holidays = [datetime.date(2018, 7, 4), datetime.date(2018, 9, 3)]
weekday = stringsToDatetime(mfw18_scheduled_departures_24h)
saturday = stringsToDatetime(satw18_scheduled_departures_24h)
sunday = stringsToDatetime(sunw18_scheduled_departures_24h)

startingDate = datetime.date(2018, 1, 7)

In [142]:
timestamps_2018_2 = generate_more_timestamps(startingDate, holidays, weekday, saturday, sunday,
                                             [pd.Timestamp(2018, 1, 7, 0, 0), pd.Timestamp(2018, 1, 7, 1, 25)])

In [143]:
timestamps_2018 = (timestamps_2018 + timestamps_2018_2)[:5726]

In [144]:
len(timestamps_2018)

5726

In [145]:
bainbridge_departures_2018_with_index = bainbridge_departures_n13_n14_n15_n16_n17.reset_index(drop=True)
bainbridge_departures_2018_with_index.index = pd.to_datetime(timestamps_2018)
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index["2018"]

In [146]:
verify_timestamps(bainbridge_departures_2018_with_index)

Was able to match departure times up until this row at index 2018-05-28 10:25:00:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       45:00.0
actual_departure          45:21.0
Actual_Arrival            16:17.0
TransitTime                    31
Name: 2018-05-28 10:25:00, dtype: object


In [147]:
bainbridge_departures_2018_with_index["2018-05-28 08:25":"2018-05-28 11:30"]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2018-05-28 08:45:00,Bainbridge,Colman,45:00.0,46:34.0,21:55.0,35.0
2018-05-28 09:40:00,Bainbridge,NaN,40:00.0,NaN,NaN,NaN
2018-05-28 10:25:00,Bainbridge,Colman,45:00.0,45:21.0,16:17.0,31.0
2018-05-28 11:30:00,Bainbridge,Colman,25:00.0,26:06.0,59:12.0,33.0


`2018-05-28 10:25:00` is an erroneous entry

In [148]:
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index.drop(pd.datetime(2018, 5, 28, 10, 25))

In [149]:
bainbridge_departures_2018_with_index["2018-05-28 08:25":"2018-05-28 11:30"]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2018-05-28 08:45:00,Bainbridge,Colman,45:00.0,46:34.0,21:55.0,35.0
2018-05-28 09:40:00,Bainbridge,NaN,40:00.0,NaN,NaN,NaN
2018-05-28 11:30:00,Bainbridge,Colman,25:00.0,26:06.0,59:12.0,33.0


In [150]:
len(timestamps_2018)

5726

In [151]:
bainbridge_departures_2018_with_index.index = pd.to_datetime(timestamps_2018[:5725])
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index["2018"]

In [152]:
verify_timestamps(bainbridge_departures_2018_with_index)

Was able to match departure times up until this row at index 2018-05-28 12:20:00:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       35:00.0
actual_departure          36:14.0
Actual_Arrival            07:55.0
TransitTime                    31
Name: 2018-05-28 12:20:00, dtype: object


In [153]:
bainbridge_departures_2018_with_index["2018-05-28":"2018-05-29"]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2018-05-28 00:00:00,Bainbridge,Colman,00:00.0,28:17.0,58:27.0,30.0
2018-05-28 01:25:00,Bainbridge,Colman,25:00.0,50:13.0,20:37.0,30.0
2018-05-28 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2018-05-28 05:20:00,Bainbridge,NaN,20:00.0,20:58.0,NaN,NaN
2018-05-28 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
2018-05-28 07:05:00,Bainbridge,NaN,05:00.0,05:08.0,NaN,NaN
2018-05-28 07:55:00,Bainbridge,NaN,55:00.0,02:42.0,NaN,NaN
2018-05-28 08:45:00,Bainbridge,Colman,45:00.0,46:34.0,21:55.0,35.0
2018-05-28 09:40:00,Bainbridge,NaN,40:00.0,NaN,NaN,NaN
2018-05-28 10:25:00,Bainbridge,Colman,25:00.0,26:06.0,59:12.0,33.0


`[2018-05-28 12:20:00, 2018-05-28 14:55:00, 2018-05-28 16:45:00, 2018-05-28 18:40:00, 2018-05-28 20:20:00, 2018-05-28 21:00:00, 2018-05-29 00:55:00, 2018-05-29 05:20:00, 2018-05-29 07:05:00, 2018-05-29 08:45:00, 2018-05-29 10:25:00, 2018-05-29 12:20:00, 2018-05-29 14:05:00, 2018-05-29 15:50:00]` are all erroneous. What is up with this day?

In [154]:
to_drop = pd.to_datetime(['2018-05-28 12:20:00', '2018-05-28 14:55:00', '2018-05-28 16:45:00', '2018-05-28 18:40:00', 
                '2018-05-28 20:20:00', '2018-05-28 21:00:00', '2018-05-29 00:55:00', '2018-05-29 05:20:00', 
                '2018-05-29 07:05:00', '2018-05-29 08:45:00', '2018-05-29 10:25:00', '2018-05-29 12:20:00', 
                '2018-05-29 14:05:00', '2018-05-29 15:50:00'])

In [155]:
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index.drop(to_drop)

In [156]:
bainbridge_departures_2018_with_index.index = pd.to_datetime(timestamps_2018[:5711])
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index["2018"]

In [157]:
verify_timestamps(bainbridge_departures_2018_with_index)

Was able to match departure times up until this row at index 2018-08-25 00:55:00:
departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure       00:00.0
actual_departure          02:13.0
Actual_Arrival            33:37.0
TransitTime                    31
Name: 2018-08-25 00:55:00, dtype: object


In [158]:
bainbridge_departures_2018_with_index["2018-08-25"]

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
2018-08-25 00:55:00,Bainbridge,Colman,00:00.0,02:13.0,33:37.0,31.0
2018-08-25 05:20:00,Bainbridge,Colman,55:00.0,25:17.0,56:25.0,31.0
2018-08-25 07:05:00,Bainbridge,Colman,20:00.0,19:29.0,51:22.0,32.0
2018-08-25 07:55:00,Bainbridge,Colman,05:00.0,05:33.0,37:39.0,32.0
2018-08-25 08:45:00,Bainbridge,Colman,55:00.0,55:24.0,26:00.0,31.0
2018-08-25 09:45:00,Bainbridge,Colman PO,45:00.0,NaN,19:06.0,NaN
2018-08-25 10:25:00,Bainbridge,Colman,45:00.0,47:37.0,18:06.0,31.0
2018-08-25 11:35:00,Bainbridge,Colman PO,25:00.0,NaN,58:39.0,NaN
2018-08-25 12:20:00,Bainbridge,Colman,35:00.0,35:00.0,05:14.0,30.0
2018-08-25 13:20:00,Bainbridge,Colman PO,20:00.0,NaN,55:40.0,NaN


`2018-08-25 00:55:00` looks erroneous

In [159]:
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index.drop(pd.to_datetime('2018-08-25 00:55:00'))

In [160]:
bainbridge_departures_2018_with_index.index = pd.to_datetime(timestamps_2018[:5710])
bainbridge_departures_2018_with_index = bainbridge_departures_2018_with_index["2018"]

In [161]:
verify_timestamps(bainbridge_departures_2018_with_index)

Success!


## Writing Everything to CSV

In [163]:
bainbridge_departures_2013_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2013_Indexed.csv")
bainbridge_departures_2014_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2014_Indexed.csv")
bainbridge_departures_2015_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2015_Indexed.csv")
bainbridge_departures_2016_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2016_Indexed.csv")
bainbridge_departures_2017_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2017_Indexed.csv")
bainbridge_departures_2018_with_index.to_csv("../csv-files/Step 1 - Indexed/Bainbridge_Departures_2018_Indexed.csv")